In [3]:
import scanpy as sc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import anndata as an
import numpy as np

MERFISH_DATA_PATH = "/mnt/merfish18/SenNet/objects/SenTMLiv_126to127_age0416.h5ad"
MERFISH_DATA_BATCH_NAMES = ['M126','M127']

SENESCENCE_GENES_PATH = "sen_genes/senscore_genes_best.csv"

REFERENCE_DATA_PATH = "/mnt/merfish18/SenNet/References/240417_02_trip_liver_filt_ct_clust_batch_anno_rna_only_v3.h5ad"

# Output variables
OUTPUT_DIRECTORY = ""
HI_DEF_DPI = 1000
# TODO: Scale function to get spatial embedding dots close to actuall cell size

In [4]:
# Read in data
mfdata = sc.read_h5ad(MERFISH_DATA_PATH)
refdata = sc.read(REFERENCE_DATA_PATH)
print(mfdata)
print(refdata)

PermissionError: [Errno 13] Permission denied: '/mnt/merfish18/SenNet/References/240417_02_trip_liver_filt_ct_clust_batch_anno_rna_only_v3.h5ad'

In [ ]:
var_intersect = mfdata.var_names.intersection(refdata.var_names)
mfdata = mfdata[mfdata.obs['batch'].isin(MERFISH_DATA_BATCH_NAMES), var_intersect].copy()
refdata = refdata[:, var_intersect].copy()

In [19]:
sc.pp.filter_cells(mfdata, min_genes=5)
sc.pp.filter_genes(mfdata, min_cells=1)

sc.pp.normalize_total(mfdata)
sc.pp.log1p(mfdata)

/home/eboone/anaconda3/envs/CellSenescence/lib/python3.12/site-packages/scanpy/preprocessing/_normalization.py:233: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))
/home/eboone/anaconda3/envs/CellSenescence/lib/python3.12/site-packages/scanpy/preprocessing/_simple.py:380: RuntimeWarning: invalid value encountered in log1p
  np.log1p(X, out=X)


In [27]:
adata = an.concat([mfdata, refdata], label='source')
print(adata)

AnnData object with n_obs × n_vars = 577462 × 299
    obs: 'celltype', 'age', 'source'
    obsm: 'X_pca'


In [25]:
print(adata.obs)

                                                  fov_y        fov_x  fov  \
2-M126-M126                                  117.237576  1909.425663    0   
10001-M126-M126                             1036.398096  1714.786181    1   
50175-M126-M126                              945.711303  1055.879459    5   
140009-M126-M126                            1391.738114    70.821114   14   
140017-M126-M126                            1455.009553   476.014895   14   
...                                                 ...          ...  ...   
QY_2326_1_2_QY_2325_1_2_TTGTGCGAGTTGTCAA-1          NaN          NaN  NaN   
QY_2326_1_2_QY_2325_1_2_GAGTCAAAGGCACAGG-1          NaN          NaN  NaN   
QY_2326_1_2_QY_2325_1_2_GGTACTTAGCGCTAAT-1          NaN          NaN  NaN   
QY_2326_1_2_QY_2325_1_2_AAGGTATAGGCCGGAA-1          NaN          NaN  NaN   
QY_2326_1_2_QY_2325_1_2_AATTAGCGTTAAGGTT-1          NaN          NaN  NaN   

                                               global_x    global_y   volum

In [28]:
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

ValueError: Input contains NaN.

In [ ]:
sc.pl.umap(adata, color="batch")

In [ ]:
sc.external.pp.harmony_integrate(adata, key="batch", max_iter_harmony=20)

In [ ]:
sc.pp.neighbors(adata, use_rep="X_pca_harmony")
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color="batch")

In [ ]:
sc.pl.umap(adata, color="cluster_names")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
traindata = adata[adata.obs["batch"] == "1"] 
nn = KNeighborsClassifier(n_jobs=16)
nn.fit(traindata.obsm["X_pca_harmony"], traindata.obs["cluster_names"])
pred = nn.predict(adata[adata.obs["batch"] == "0"].obsm["X_pca_harmony"])

In [ ]:
adata.obs.loc[adata.obs["batch"] == "0", "cluster_names"] = pred

In [ ]:
sc.pl.umap(adata, color="cluster_names")

In [ ]:
# Attach celltype and age labels to each cell
# Celltype label
mfdata.obs["celltype"] = list(adata[adata.obs["batch"] == "0"].obs["cluster_names"])

# Age label
get_age = lambda x : 'young' if x < -2400 else 'old'
mfdata.obs["age"] = [get_age(x) for x in mfdata.obsm['X_spatial'][:, 0]]

In [ ]:
sc.set_figure_params(dpi=1000)
sc.pl.embedding(mfdata, basis="X_spatial", color="celltype", frameon=False, title="")#, groups=["Kupffer cell"])
sc.pl.embedding(mfdata, basis="X_spatial", color="age", frameon=False, title="")#, groups=["Kupffer cell"])

In [ ]:
mfdata.obsm["X_spatial"].min(axis=0), mfdata.obsm["X_spatial"].max(axis=0)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(mfdata.obsm["X_spatial"][:,0], mfdata.obsm["X_spatial"][:,1])

In [ ]:
sc.pl.embedding(mfdata[(mfdata.obsm["X_spatial"][:,0] > 1500) & (mfdata.obsm["X_spatial"][:,0] < 2500) & (mfdata.obsm["X_spatial"][:,1] > -1500) & (mfdata.obsm["X_spatial"][:,1] < 1500)], basis="X_spatial", color="celltype", frameon=False, title="")#, groups=["Kupffer cell"])

In [ ]:
mfdata.write("M126_test.h5ad")

In [ ]:
for celltype in mfdata.obs["celltype"].unique():
    sc.pl.embedding(mfdata[(mfdata.obsm["X_spatial"][:,0] > -1500) & (mfdata.obsm["X_spatial"][:,0] < 1500) & (mfdata.obsm["X_spatial"][:,1] > -1500) & (mfdata.obsm["X_spatial"][:,1] < 1500)], basis="X_spatial", color="celltype", frameon=False, title=celltype, groups=[celltype], legend_loc="off")

In [ ]:
sc.pl.embedding(mfdata[(mfdata.obsm["X_spatial"][:,0] > -1500) & (mfdata.obsm["X_spatial"][:,0] < 1500) & (mfdata.obsm["X_spatial"][:,1] > -1500) & (mfdata.obsm["X_spatial"][:,1] < 1500)], basis="X_spatial", color="celltype", frameon=False, title="10_hepatocyte", groups=["10_hepatocyte", "4_hepatocyte", "1_hepatocyte"], palette=["tab:blue", "tab:orange", "tab:green", "tab:red"], na_in_legend=False)

In [ ]:
adata1 = sc.read_h5ad("not/a/real/file")
adata2 = sc.read_h5ad("not/a/real/file")
print(adata1, adata2)
print(set(adata1.obs['batch']), set(adata1.obs['celltype']))
print(set(adata2.obs['batch']), set(adata2.obs['celltype']))

comb = adata1.concatenate(adata2, batch_key='ignore_batch', index_unique=None)
print(set(comb.obs['batch']))
print(comb.obs['celltype'])
print(comb)

comb.write_h5ad("data_raw/M126to127_test2.h5ad")